In [157]:
# Extensions
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [1]:
# Imports
import torch
import pandas as pd
import numpy as np

In [2]:
# Configuration
ID = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x = torch.randn(10, 4, 100)
x_rev_comp = torch.randn(10, 4, 100)
y = torch.randint(0, 2, (10, 1))
batch = (ID, x, x_rev_comp, y)

In [3]:
# Helper functions
def move_module_to_cpu(module):
    module.to("cpu")
    for child in module.children():
        move_module_to_cpu(child)

# Towers

In [4]:
from eugene.models.base import _towers as towers

### Feed Forward Tower

In [5]:
tower = towers.Tower(
    input_size=400,
    block=torch.nn.Linear,
    repeats=3,
    dynamic_block_args={'in_features': [400, 200, 100], 'out_features': [200, 100, 10]},
)
tower_out = tower(x.reshape(10, -1))
tower.input_size, tower.output_size, tower_out.shape

(400, [10], torch.Size([10, 10]))

### Tower of Conv1D blocks

In [6]:
from eugene.models.base._blocks import Conv1DBlock

In [16]:
tower = towers.Tower(
    input_size=(4, 100),
    block=Conv1DBlock,
    repeats=3,
    static_block_args={'input_len': 100, 'conv_kernel': 3, 'conv_padding': 'same', 'conv_type': 'conv1d', 'order': 'conv-norm-act-dropout-pool'},
    dynamic_block_args={'input_channels': [4, 10, 20], 'output_channels': [10, 20, 30]}
)
tower_out = tower(x)
tower, tower.input_size, tower.output_size, tower_out.shape

(Tower(
   (blocks): Sequential(
     (conv1dblock_0): Conv1DBlock(
       (layers): Sequential(
         (conv): Conv1d(4, 10, kernel_size=(3,), stride=(1,), padding=same)
         (norm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act): ReLU()
         (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
       )
     )
     (conv1dblock_1): Conv1DBlock(
       (layers): Sequential(
         (conv): Conv1d(10, 20, kernel_size=(3,), stride=(1,), padding=same)
         (norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act): ReLU()
         (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
       )
     )
     (conv1dblock_2): Conv1DBlock(
       (layers): Sequential(
         (conv): Conv1d(20, 30, kernel_size=(3,), stride=(1,), padding=same)
         (norm): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [17]:
tower = towers.Tower(
    input_size=(4, 100),
    block=Conv1DBlock,
    repeats=3,
    static_block_args={'input_len': 100, 'conv_kernel': 3, 'conv_padding': 'same', 'conv_type': 'conv1d', 'activation': 'gelu', 'order': 'conv-norm-act-dropout-pool'},
    dynamic_block_args={'input_channels': [4, 10, 20], 'output_channels': [10, 20, 30]},
    mults={"conv_dilation": 2}
)
tower_out = tower(x)
tower, tower.input_size, tower.output_size, tower_out.shape

(Tower(
   (blocks): Sequential(
     (conv1dblock_0): Conv1DBlock(
       (layers): Sequential(
         (conv): Conv1d(4, 10, kernel_size=(3,), stride=(1,), padding=same)
         (norm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act): GELU()
         (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
       )
     )
     (conv1dblock_1): Conv1DBlock(
       (layers): Sequential(
         (conv): Conv1d(10, 20, kernel_size=(3,), stride=(1,), padding=same, dilation=(2,))
         (norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act): GELU()
         (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
       )
     )
     (conv1dblock_2): Conv1DBlock(
       (layers): Sequential(
         (conv): Conv1d(20, 30, kernel_size=(3,), stride=(1,), padding=same, dilation=(4,))
         (norm): BatchNorm1d(30, eps=1e-05, momentum=0.1, a

### Conv1DTower
Allows for more explicit definition of a tower of convolutions. Will be deprecated, but currently used to work with models.

In [18]:
conv1d_tower = towers.Conv1DTower(
    input_len=100,
    input_channels=4,
    conv_channels=[10, 10],
    conv_kernels=[3, 3],
    conv_strides=[1, 1],
    conv_dilations=[1, 1],
    conv_padding=["valid", "valid"],
    activations=[None, "relu"],
    pool_types="avg",
    pool_kernels=[1, 1],
    pool_strides=[1, 1],
    pool_padding=[0, 0],
    dropout_rates=[None, 0.5],
    batchnorm=True,
    batchnorm_first=True
)
tower_out = conv1d_tower(x)
conv1d_tower, tower_out.shape

(Conv1DTower(
   (layers): Sequential(
     (0): Conv1d(4, 10, kernel_size=(3,), stride=(1,), padding=valid)
     (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): AvgPool1d(kernel_size=1, stride=1, padding=(0,))
     (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
     (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (5): ReLU()
     (6): AvgPool1d(kernel_size=1, stride=1, padding=(0,))
     (7): Dropout(p=0.5, inplace=False)
   )
 ),
 torch.Size([10, 10, 96]))

### BiConv1DTower

In [20]:
biconv1d_tower = towers.BiConv1DTower(
    filters = 10,
    kernel_size = 3,
    layers = 2
)
tower_out = biconv1d_tower(x)
biconv1d_tower, tower_out.shape

(BiConv1DTower(
   (kernels): ParameterList(
       (0): Parameter containing: [torch.float32 of size 10x4x3]
       (1): Parameter containing: [torch.float32 of size 10x10x3]
   )
   (biases): ParameterList(
       (0): Parameter containing: [torch.float32 of size 10]
       (1): Parameter containing: [torch.float32 of size 10]
   )
 ),
 torch.Size([10, 10, 100]))

# DONE

---

# Scratch